In [ ]:
#uses census API to fetch the count of people with certain attributes in a given county for a given year
#then, converts those counts to population proportions

import requests
import pandas as pd
import sys
import os

API_KEY = os.getenv("CENSUS_API_KEY")
years = range(2013, 2023)  # 2013 to 2022 inclusive
dfs = []
url_template = "https://api.census.gov/data/{year}/acs/acs5"

# Updated base variable list (without geography fields)
base_var_list = [
    "NAME",
    # Sex:
    "B01001_002E",  # Male count
    "B01001_026E",  # Female count
    # Disability:
    "B18101_001E",  # Total for disability status
    "B18101_004E",  # Male, Under 5, with a disability
    "B18101_007E",  # Male, 5 to 17, with a disability
    "B18101_010E",  # Male, 18 to 34, with a disability
    "B18101_013E",  # Male, 35 to 64, with a disability
    "B18101_016E",  # Male, 65 to 74, with a disability
    "B18101_019E",  # Male, 75 and over, with a disability
    "B18101_023E",  # Female, Under 5, with a disability
    "B18101_026E",  # Female, 5 to 17, with a disability
    "B18101_029E",  # Female, 18 to 34, with a disability
    "B18101_032E",  # Female, 35 to 64, with a disability
    "B18101_035E",  # Female, 65 to 74, with a disability
    "B18101_038E",  # Female, 75 and over, with a disability
    # Hispanic ethnicity:
    "B03001_001E",  # Total for Hispanic origin question
    "B03001_003E",  # Hispanic or Latino
    # Race (alone):
    "B02001_002E",  # White alone
    "B02001_003E",  # Black or African American alone
    "B02001_004E",  # American Indian and Alaska Native alone
    "B02001_005E",  # Asian alone
    "B02001_006E",  # Native Hawaiian and Other Pacific Islander alone
    # Veteran:
    "B21001_002E",  # Veteran count
    # Total population (for veteran proportion):
    "B01003_001E",
    # Education (for population 25+):
    # Less than HS Diploma: B15003_002E to B15003_016E
    "B15003_002E", "B15003_003E", "B15003_004E", "B15003_005E", "B15003_006E", 
    "B15003_007E", "B15003_008E", "B15003_009E", "B15003_010E", "B15003_011E", 
    "B15003_012E", "B15003_013E", "B15003_014E", "B15003_015E", "B15003_016E",
    # High School Diploma Level: B15003_017E to B15003_020E
    "B15003_017E", "B15003_018E", "B15003_019E", "B15003_020E",
    # College+ Degree: B15003_021E to B15003_025E
    "B15003_021E", "B15003_022E", "B15003_023E", "B15003_024E", "B15003_025E",
    # Median Income:
    "B19013_001E"
]
var_str = ",".join(base_var_list)

for year in years:
    url = url_template.format(year=year)
    params = {
        "get": var_str,
        "for": "county:*",  # All counties in Georgia
        "in": "state:13",
        "key": API_KEY
    }
    try:
        response = requests.get(url, params=params, timeout=40)
        response.raise_for_status()
        data = response.json()
        df_year = pd.DataFrame(data[1:], columns=data[0])
        
        # Convert all numeric columns (all except "NAME") to numbers.
        numeric_cols = [col for col in base_var_list if col != "NAME"]
        for col in numeric_cols:
            df_year[col] = pd.to_numeric(df_year[col], errors='coerce')
        
        # The API automatically appends "state" and "county". Drop "state" and rename "county" to "code".
        if "state" in df_year.columns:
            df_year.drop(columns=["state"], inplace=True)
        if "county" in df_year.columns:
            df_year.rename(columns={"county": "code"}, inplace=True)
        
        # Create new raw count columns with the year prefix.
        # Sex:
        df_year[f"{year}_male_count"] = df_year["B01001_002E"]
        df_year[f"{year}_female_count"] = df_year["B01001_026E"]
        
        # Disability: Sum the cells representing "with a disability" (both sexes & all age groups)
        df_year[f"{year}_disability_yes_count"] = (
            df_year["B18101_004E"] + df_year["B18101_007E"] + df_year["B18101_010E"] +
            df_year["B18101_013E"] + df_year["B18101_016E"] + df_year["B18101_019E"] +
            df_year["B18101_023E"] + df_year["B18101_026E"] + df_year["B18101_029E"] +
            df_year["B18101_032E"] + df_year["B18101_035E"] + df_year["B18101_038E"]
        )
        df_year[f"{year}_disability_no_count"] = df_year["B18101_001E"] - df_year[f"{year}_disability_yes_count"]
        
        # Hispanic:
        df_year[f"{year}_hispanic_count"] = df_year["B03001_003E"]
        df_year[f"{year}_nonhispanic_count"] = df_year["B03001_001E"] - df_year["B03001_003E"]
        
        # Race:
        df_year[f"{year}_race_white_count"] = df_year["B02001_002E"]
        df_year[f"{year}_race_black_count"] = df_year["B02001_003E"]
        df_year[f"{year}_race_indian_count"] = df_year["B02001_004E"]
        df_year[f"{year}_race_asian_count"] = df_year["B02001_005E"]
        df_year[f"{year}_race_PI_count"] = df_year["B02001_006E"]
        
        # Veteran:
        df_year[f"{year}_veteran_count"] = df_year["B21001_002E"]
        df_year[f"{year}_pop_count"] = df_year["B01003_001E"]
        
        # Education:
        edu_less_cols = [
            "B15003_002E", "B15003_003E", "B15003_004E", "B15003_005E", "B15003_006E",
            "B15003_007E", "B15003_008E", "B15003_009E", "B15003_010E", "B15003_011E",
            "B15003_012E", "B15003_013E", "B15003_014E", "B15003_015E", "B15003_016E"
        ]
        edu_hs_cols = [
            "B15003_017E", "B15003_018E", "B15003_019E", "B15003_020E"
        ]
        edu_college_cols = [
            "B15003_021E", "B15003_022E", "B15003_023E", "B15003_024E", "B15003_025E"
        ]
        df_year[f"{year}_less_hs_degree_count"] = df_year[edu_less_cols].sum(axis=1)
        df_year[f"{year}_hs_degree_count"] = df_year[edu_hs_cols].sum(axis=1)
        df_year[f"{year}_college_degree_count"] = df_year[edu_college_cols].sum(axis=1)
        
        # Median Income:
        df_year[f"{year}_median_income"] = df_year["B19013_001E"]
        
        # Select only the new raw count columns plus "code"
        cols_to_keep = [
            f"{year}_male_count", f"{year}_female_count",
            f"{year}_disability_yes_count", f"{year}_disability_no_count",
            f"{year}_hispanic_count", f"{year}_nonhispanic_count",
            f"{year}_race_white_count", f"{year}_race_black_count",
            f"{year}_race_indian_count", f"{year}_race_asian_count",
            f"{year}_race_PI_count", f"{year}_veteran_count",
            f"{year}_pop_count",
            f"{year}_less_hs_degree_count", f"{year}_hs_degree_count",
            f"{year}_college_degree_count",
            f"{year}_median_income",
            "code"
        ]
        df_year = df_year[["NAME"] + cols_to_keep]
        df_year.set_index("NAME", inplace=True)
        
        dfs.append(df_year)
        print(f"Data for {year} fetched and processed successfully.")
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {year}: {e}")

# Merge all yearly DataFrames on the index (county name)
if dfs:
    df_final = dfs[0]
    for df in dfs[1:]:
        df_final = df_final.join(df.drop(columns=["code"]), how="outer")
    
    # Calculate proportions for each year (separately for each county)
    for year in years:
        # Sex proportions
        male = df_final[f"{year}_male_count"]
        female = df_final[f"{year}_female_count"]
        total_sex = male + female
        df_final[f"{year}_male_prop"] = male / total_sex
        df_final[f"{year}_female_prop"] = female / total_sex

        # Disability proportions
        dis_yes = df_final[f"{year}_disability_yes_count"]
        dis_no = df_final[f"{year}_disability_no_count"]
        total_dis = dis_yes + dis_no
        df_final[f"{year}_disability_yes_prop"] = dis_yes / total_dis
        df_final[f"{year}_disability_no_prop"] = dis_no / total_dis

        # Hispanic proportions
        hispanic = df_final[f"{year}_hispanic_count"]
        nonhispanic = df_final[f"{year}_nonhispanic_count"]
        total_hisp = hispanic + nonhispanic
        df_final[f"{year}_hispanic_prop"] = hispanic / total_hisp
        df_final[f"{year}_nonhispanic_prop"] = nonhispanic / total_hisp

        # Race proportions
        white = df_final[f"{year}_race_white_count"]
        black = df_final[f"{year}_race_black_count"]
        indian = df_final[f"{year}_race_indian_count"]
        asian = df_final[f"{year}_race_asian_count"]
        pi = df_final[f"{year}_race_PI_count"]
        total_race = white + black + indian + asian + pi
        df_final[f"{year}_race_white_prop"] = white / total_race
        df_final[f"{year}_race_black_prop"] = black / total_race
        df_final[f"{year}_race_indian_prop"] = indian / total_race
        df_final[f"{year}_race_asian_prop"] = asian / total_race
        df_final[f"{year}_race_PI_prop"] = pi / total_race

        # Veteran proportion (using total population)
        veteran = df_final[f"{year}_veteran_count"]
        pop_count = df_final[f"{year}_pop_count"]
        df_final[f"{year}_veteran_prop"] = veteran / pop_count

        # Education proportions (for population 25+)
        less = df_final[f"{year}_less_hs_degree_count"]
        hs = df_final[f"{year}_hs_degree_count"]
        college = df_final[f"{year}_college_degree_count"]
        edu_total = less + hs + college
        df_final[f"{year}_less_hs_prop"] = less / edu_total
        df_final[f"{year}_hs_prop"] = hs / edu_total
        df_final[f"{year}_college_prop"] = college / edu_total

    # --- Remove the Raw Count Columns; Keep Only Proportion Columns, Median Income, and "code" ---
    # Retain any column that ends with '_prop', '_median_income', or is "code"
    keep_cols = [col for col in df_final.columns if col.endswith('_prop') or col.endswith('_median_income') or col == 'code']
    df_final = df_final[keep_cols]
    
    output_filename = "/Users/ianpoe/Desktop/ga_county_proportions_2013_2022.csv"
    df_final.to_csv(output_filename)
    print(f"\nFinal merged DataFrame (only proportions and median income) exported to {output_filename}")
    print(df_final)
else:
    print("No data fetched for any year.")



Data for 2013 fetched and processed successfully.
Data for 2014 fetched and processed successfully.
Data for 2015 fetched and processed successfully.
Data for 2016 fetched and processed successfully.
Data for 2017 fetched and processed successfully.
Data for 2018 fetched and processed successfully.
Data for 2019 fetched and processed successfully.
Data for 2020 fetched and processed successfully.
Data for 2021 fetched and processed successfully.
Data for 2022 fetched and processed successfully.


/var/folders/t1/f9l59kf14m104mc7sxz3dzyc0000gn/T/ipykernel_24355/2043120649.py:190: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f"{year}_race_black_prop"] = black / total_race
/var/folders/t1/f9l59kf14m104mc7sxz3dzyc0000gn/T/ipykernel_24355/2043120649.py:191: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f"{year}_race_indian_prop"] = indian / total_race
/var/folders/t1/f9l59kf14m104mc7sxz3dzyc0000gn/T/ipykernel_24355/2043120649.py:192: PerformanceWarning: DataFrame is highly fragmented.  This is usually the r


Final merged DataFrame (only proportions and median income) exported to /Users/ianpoe/Desktop/ga_county_proportions_2013_2022.csv
                           2013_median_income code  2014_median_income  \
NAME                                                                     
Appling County, Georgia                 36786  001               38461   
Atkinson County, Georgia                30049  003               30403   
Bacon County, Georgia                   33757  005               37698   
Baker County, Georgia                   27353  007               45526   
Baldwin County, Georgia                 32751  009               31758   
...                                       ...  ...                 ...   
Whitfield County, Georgia               40471  313               40081   
Wilcox County, Georgia                  32928  315               32545   
Wilkes County, Georgia                  28983  317               30729   
Wilkinson County, Georgia               36173  319     

In [24]:
#performs stratified sampling on a dataset of interest. Uses population proportions for covariates to
#capture a sample of dataset entries such that the sample proportions are a good match with the population proportions, overall

import pandas as pd
import numpy as np

def sample_to_match_pop(df, pop_props, sample_size, random_state=None):
    """
    Select a weighted random sample from `df` such that the weighted covariate
    distributions match the target population proportions.
    
    Parameters:
      df: pandas DataFrame with the nonprofit-level data.
      pop_props: dict; keys are column names (e.g., 'race', 'gender') and values are
                 dicts mapping each category to its population proportion.
                 Example:
                 {
                   'race': {'asian': 0.35, 'black': 0.20, 'white': 0.45},
                   'gender': {'male': 0.50, 'female': 0.50}
                 }
      sample_size: int; desired number of rows in the final sample.
      random_state: int or None; for reproducibility.
      
    Returns:
      A pandas DataFrame containing the sampled rows.
    """
    
    # Initialize weights as 1 for every observation.
    weights = np.ones(len(df))
    
    # Loop through each covariate in pop_props.
    for covariate, target_props in pop_props.items():
        # Compute observed proportions in the nonprofit data for the current covariate.
        observed = df[covariate].value_counts(normalize=True).to_dict()
        
        # For each observation, compute the weight factor as:
        #   (target population proportion) / (observed nonprofit proportion)
        # for the category that observation falls into.
        def weight_factor(value):
            if value in target_props and value in observed:
                return target_props[value] / observed[value]
            else:
                # If a category is not provided in the target or is missing in the data,
                # leave the weight unchanged.
                return 1.0
        
        # Multiply the current weights by the weight factors for this covariate.
        weights *= df[covariate].apply(weight_factor).values
    
    # Normalize the weights so they sum to 1.
    weights = weights / weights.sum()
    
    # Sample from the DataFrame using the computed weights.
    sampled_df = df.sample(n=sample_size, weights=weights, random_state=random_state)
    return sampled_df

# Example usage:
# Assume df is your nonprofit-level pandas DataFrame containing columns 'race' and 'gender'
# and you have the following target population proportions:
pop_props = {
    'race': {'asian': 0.35, 'black': 0.20, 'white': 0.45},
    'gender': {'male': 0.50, 'female': 0.50}
}

# Specify desired sample size (e.g., 1000 rows).
sample_size = 1000

# Draw the sample.
sampled_df = sample_to_match_pop(df, pop_props, sample_size=sample_size, random_state=42)

# Verify balance by comparing distributions:
print("Sampled race distribution:")
print(sampled_df['race'].value_counts(normalize=True))
print("\nTarget race distribution:")
print(pd.Series(pop_props['race']))

print("\nSampled gender distribution:")
print(sampled_df['gender'].value_counts(normalize=True))
print("\nTarget gender distribution:")
print(pd.Series(pop_props['gender']))


KeyError: 'race'

In [ ]:
#using census API, fetches population for a given county in a given year

API_KEY = os.getenv("CENSUS_API_KEY")
years = range(2013, 2023)  # 2013 to 2022 inclusive
dfs = []
url_template = "https://api.census.gov/data/{year}/acs/acs5"

for year in years:
    url = url_template.format(year=year)
    params = {
        "get": "NAME,B01003_001E",  # Total population
        "for": "county:*",         # All counties in Georgia
        "in": "state:13",
        "key": API_KEY
    }
    try:
        response = requests.get(url, params=params, timeout=20)
        response.raise_for_status()
        data = response.json()
        df_year = pd.DataFrame(data[1:], columns=data[0])
        
        # Rename columns:
        # - B01003_001E -> <year>_pop (Total Population)
        # - Rename "county" to "code" for consistency.
        df_year.rename(columns={
            "B01003_001E": f"{year}_pop",
            "county": "code"
        }, inplace=True)
        
        # Drop the "state" column (automatically appended by the API)
        if "state" in df_year.columns:
            df_year.drop(columns=["state"], inplace=True)
        
        # Convert the population column to numeric
        df_year[f"{year}_pop"] = pd.to_numeric(df_year[f"{year}_pop"], errors='coerce')
        
        # Set county name as the index.
        df_year.set_index("NAME", inplace=True)
        
        dfs.append(df_year)
        print(f"Data for {year} fetched successfully.")
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {year}: {e}")

# Merge all yearly DataFrames on the index (county name)
if dfs:
    df_final = dfs[0]
    for df in dfs[1:]:
        # Drop duplicate "code" column from subsequent DataFrames
        df_final = df_final.join(df.drop(columns=["code"]), how="outer")
    
    print("\nFinal merged DataFrame:")
    print(df_final)
    
    # Export the final DataFrame to CSV
    output_filename = "/Users/ianpoe/Desktop/population_by_county_2013_2022.csv"
    df_final.to_csv(output_filename)
    print(f"\nData exported to {output_filename}")
else:
    print("No data fetched for any year.")

sys.exit(0)


Data for 2013 fetched successfully.
Data for 2014 fetched successfully.
Data for 2015 fetched successfully.
Data for 2016 fetched successfully.
Data for 2017 fetched successfully.
Data for 2018 fetched successfully.
Data for 2019 fetched successfully.
Data for 2020 fetched successfully.
Data for 2021 fetched successfully.
Data for 2022 fetched successfully.

Final merged DataFrame:
                           2013_pop code  2014_pop  2015_pop  2016_pop  \
NAME                                                                     
Appling County, Georgia       18354  001     18421     18417     18410   
Atkinson County, Georgia       8332  003      8297      8294      8268   
Bacon County, Georgia         11159  005     11196     11222     11251   
Baker County, Georgia          3410  007      3342      3292      3250   
Baldwin County, Georgia       46018  009     45854     45795     45808   
...                             ...  ...       ...       ...       ...   
Whitfield County, Georg

SystemExit: 0

/opt/anaconda3/envs/machine-learning-env/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
